Chatbot gibi uygulamalar yapmak için LLM'leri kullanırıken hafıza önemli bir olaydır ve durumsal olmayan (hafızası olmayan) tokenlerle bu işlemin nasıl yapılacağını göreceğiz.
# ConversationBufferMemory (Konuşma Arabellek Hafızası)
Önceki konuşmaları da yeni promptlara ekleyip yollar. Fazla token tüketir, hafızası daha güçlüdür.

In [ ]:
# Import'lar
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lokal .env dosyasından oku

import warnings
warnings.filterwarnings('ignore')

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.0)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Selam, benim adım Harrison.")
conversation.predict(input="1+1 ne yapar?")
conversation.predict(input="Benim ismim nedir?")
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})
memory = ConversationBufferMemory()
memory.save_context({"input": "Selam"},
                    {"output": "Naber?"})
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})
memory.save_context({"input": "Hiç, öyle takılıyorum."},
                    {"output": "İyii."})
memory.load_memory_variables({})

# ConversationBufferWindowMemory (Konuşma Arabellek Penceresi Hafızası)
Verilen k sayısına kadar önceki konuşmaları yollar böylece belli bir konuşma öncesini unutmaya başlar.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Selam"},
                    {"output": "Naber"})
memory.save_context({"input": "Hiç, sadece taklılıyorum."},
                    {"output": "İyii."})
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0)

memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Selam, benim adım Harrison.")
conversation.predict(input="1+1 ne yapar?")
conversation.predict(input="Benim ismim nedir?")

# ConversationTokenBufferMemory (Konuşma Tokeni Arabellek Hafızası)
Konuşma token arabellek hafızası, bellekte son etkileşimlerin bir arabelleğini tutar ve etkileşimlerin temizlenip temizlenmeyeceğini belirlemek için etkileşim sayısı yerine token uzunluğunu kullanır.

In [ ]:
#!pip install tiktoken
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

memory.load_memory_variables({})

# ConversationSummaryMemory (Konuşma Özet Arabellek Hafızası)
max_token_limit (maksimum token limiti) değeri önceki stringleri verilen değere (bu örnekte 100) karakterli bir özeti şeklinde yolluyor.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

# uzun bir string oluştur
schedule = "Ürün ekibinizle sabah 8'de bir toplantı var. \
PowerPoint sunumunuzun hazırlanmasına ihtiyacınız olacak. \
09:00-12:00 saatleri arasında LangChain projeniz üzerinde çalışmak için \
zamanınız olacak ve bu, Langchain çok güçlü bir araç olduğu için hızlı bir \
şekilde ilerleyecektir. Öğlen, yapay zekadaki en son gelişmeleri anlamak için \
sizinle buluşmak üzere bir saatten fazla uzaklıktan arabayla gelen bir \
müşteriyle İtalyan restoranında öğle yemeği. \
En son LLM demosunu göstermek için dizüstü bilgisayarınızı getirdiğinizden emin olun."

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Selam"}, {"output": "Naber"})
memory.save_context({"input": "Hiç, öyle takılıyorum"},
                    {"output": "İyii"})
memory.save_context({"input": "Bugün programda neler var?"},
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Göstermek için iyi bir demo ne olurdu?")
memory.load_memory_variables({})

# Ekstra Hafıza Tipleri
*   Vektör Data Hafızası: Vektör veritabanında metinleri tutar ve en alakalı metin bloklarını çağırır.
*   Varlık Hafızası: Bir LLM kullanarak bazı varlıkların detaylarını aklında tutar.

Birden fazla hafızayı bir arada da kullanabilirsiniz. Örneğin Konuşma Özet Belleği ile Varlık Hafızası kullanarak bireyleri hatırlamak. Aynı zamanda sohbeti bir geleneksel veritabanında da saklayabilirsiniz.